# Student Loan Risk with Deep Learning

In [1]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [2]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [3]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [4]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [5]:
# Define the target set y using the credit_ranking column
y = loans_df["credit_ranking"]

# Display a sample of y
y.head()

0    0
1    0
2    0
3    1
4    0
Name: credit_ranking, dtype: int64

In [6]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.drop("credit_ranking", axis=1)

# Review the features DataFrame
X.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


### Step 3: Split the features and target sets into training and testing datasets.


In [7]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [8]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_train_scaled = scaler.fit_transform(X_train)

# Fit the scaler to the features test dataset
X_test_scaled = scaler.transform(X_test)

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [9]:
# Define the the number of inputs (features) to the model
n_features = X_train_scaled.shape[1]

# Review the number of features
print(f"Number of input features: {n_features}")

Number of input features: 11


In [10]:
# Define the number of hidden nodes for the first hidden layer
first_layer = 100

# Define the number of hidden nodes for the second hidden layer
second_layer = 50

# Define the number of neurons in the output layer
output_layer = 1

In [11]:
# Create the Sequential model instance
model = tf.keras.models.Sequential()

# Add the first hidden layer
model.add(tf.keras.layers.Dense(units=first_layer, activation="relu", input_dim=n_features))

# Add the second hidden layer
model.add(tf.keras.layers.Dense(units=second_layer, activation="relu"))

# Add the output layer to the model specifying the number of output neurons and activation function
model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

In [12]:
# Display the Sequential model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               1200      
                                                                 
 dense_1 (Dense)             (None, 50)                5050      
                                                                 
 dense_2 (Dense)             (None, 1)                 51        
                                                                 
Total params: 6,301
Trainable params: 6,301
Non-trainable params: 0
_________________________________________________________________


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [13]:
# Compile the Sequential model
model.compile(loss="binary_crossentropy",
                    optimizer="adam",
                    metrics=["accuracy"])

In [14]:
# Fit the model using 50 epochs and the training data
fit_model = model.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
38/38 [==============================] - 1s 2ms/step - loss: 0.6338 - accuracy: 0.6614
Epoch 2/50
38/38 [==============================] - 0s 1ms/step - loss: 0.5300 - accuracy: 0.7431
Epoch 3/50
38/38 [==============================] - 0s 1ms/step - loss: 0.5095 - accuracy: 0.7540
Epoch 4/50
38/38 [==============================] - 0s 2ms/step - loss: 0.4983 - accuracy: 0.7631
Epoch 5/50
38/38 [==============================] - 0s 2ms/step - loss: 0.4852 - accuracy: 0.7656
Epoch 6/50
38/38 [==============================] - 0s 2ms/step - loss: 0.4786 - accuracy: 0.7765
Epoch 7/50
38/38 [==============================] - 0s 2ms/step - loss: 0.4683 - accuracy: 0.7765
Epoch 8/50
38/38 [==============================] - 0s 2ms/step - loss: 0.4630 - accuracy: 0.7773
Epoch 9/50
38/38 [==============================] - 0s 2ms/step - loss: 0.4575 - accuracy: 0.7857
Epoch 10/50
38/38 [==============================] - 0s 2ms/step - loss: 0.4529 - accuracy: 0.7840
Epoch 11/50
38/38 [

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [15]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Test loss: {model_loss}, Test Accuracy: {model_accuracy}")

13/13 - 0s - loss: 0.5795 - accuracy: 0.7450 - 197ms/epoch - 15ms/step
Test loss: 0.579524040222168, Test Accuracy: 0.7450000047683716


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [16]:
# Set the model's file path
file_path = "student_loans.keras"

# Export your model to a keras file
model.save(file_path)

---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [17]:
# Import the load_model function
from tensorflow.keras.models import load_model

In [18]:
# Set the model's file path
file_path = "student_loans.keras"

# Load the model to a new object
loaded_model = load_model(file_path)

### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [19]:
# Make predictions with the test data
predictions = loaded_model.predict(X_test_scaled)



13/13 [==============================] - 0s 2ms/step


### Step 3

In [20]:
# Convert them to 0 or 1 by applying a threshold of 0.5
predictions_binary = (predictions > 0.5).astype(int)

# Save the predictions to a DataFrame
predictions_df = pd.DataFrame(predictions_binary)

# Display a sample of the predictions
predictions_df

,0
0,0
1,0
2,1
3,1
4,1
...,...
395,0
396,0
397,1
398,0


### Step 4: Display a classification report with the y test data and predictions

In [21]:
# Print the classification report with the y test data and predictions
print(classification_report(y_test, predictions_binary))

              precision    recall  f1-score   support

           0       0.71      0.76      0.74       188
           1       0.78      0.73      0.75       212

    accuracy                           0.74       400
   macro avg       0.74      0.75      0.74       400
weighted avg       0.75      0.74      0.75       400



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

**1. Data includes financial background, loan preferences, educational information, and student demographics.  This data ensures the system recommends loans tailored to a student's financial situation, needs and educational path.  Government or private loans may be preferable.**


**2. Content-based filtering would be most appropriate, using attributes like student profiles and past loans.  Collaborative filtering would also help, based on what similar students have chosen.  A combination of the two filtering methods would be used based on the student's personal data and historical behavior of past students**


**3. a. Data privacy challenge.  Unauthorized access of sensitive financial and educational data requires privacy measures.  The system has to be compliant with various government regulations to main trust.  b. Bias challenge.  The training data may be biased to the advantage of certain student demographics or loan providers.  This could result in discriminatory loan recommendations.**

